In [1]:
import os, shutil

In [2]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import keras

Using TensorFlow backend.


In [4]:
import numpy as np

In [5]:
from PIL import Image

In [6]:
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
from keras.preprocessing import image

In [8]:
import time

# Create directories

In [10]:
original_dataset_dir = 'C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/train_pics'

In [11]:
base_dir = 'C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/data_small/'
#os.mkdir(base_dir)

In [12]:
train_dir = os.path.join(base_dir, 'train')
#os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
#os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
#os.mkdir(test_dir)

In [13]:
train_jump_dir = os.path.join(train_dir, 'jump')
#os.mkdir(train_jump_dir)
train_nojump_dir = os.path.join(train_dir, 'nojump')
#os.mkdir(train_nojump_dir)

In [14]:
validation_jump_dir = os.path.join(validation_dir, 'jump')
#os.mkdir(validation_jump_dir)
validation_nojump_dir = os.path.join(validation_dir, 'nojump')
#os.mkdir(validation_nojump_dir)

In [15]:
test_jump_dir = os.path.join(test_dir, 'jump')
#os.mkdir(test_jump_dir)
test_nojump_dir = os.path.join(test_dir, 'nojump')
#os.mkdir(test_nojump_dir)

In [16]:
print("total training jump images:", len(os.listdir(train_jump_dir)))
print("total training nojump images:", len(os.listdir(train_nojump_dir)))
print("total validation jump images:", len(os.listdir(validation_jump_dir)))
print("total validation nojump images:", len(os.listdir(validation_nojump_dir)))
print("total test jump images:", len(os.listdir(test_jump_dir)))
print("total test nojump images:", len(os.listdir(test_nojump_dir)))

total training jump images: 444
total training nojump images: 395
total validation jump images: 55
total validation nojump images: 81
total test jump images: 24
total test nojump images: 197


In [17]:
model = models.Sequential()
model.add(layers.Conv2D(32, (10,10), activation = "relu", input_shape=(250, 75, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (10,10), activation = "relu"))
model.add(layers.MaxPooling2D((2,2)))
#model.add(layers.Conv2D(128, (10,10), activation="relu"))
#model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (5,5), activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(1, activation = "sigmoid"))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 241, 66, 32)       9632      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 120, 33, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 111, 24, 64)       204864    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 55, 12, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 51, 8, 128)        204928    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 4, 128)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)             0         
__________

In [19]:
model.compile(loss="binary_crossentropy",
             optimizer = optimizers.RMSprop(lr=1e-4),
             metrics=["acc"])

In [29]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(250, 75),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(250, 75),
        batch_size=10,
        class_mode='binary')

Found 829 images belonging to 2 classes.
Found 146 images belonging to 2 classes.


In [30]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 250, 75, 3)
labels batch shape: (20,)


In [31]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=150,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/5
150/150 [==============================] - 216s 1s/step - loss: 0.0859 - acc: 0.9687 - val_loss: 0.1341 - val_acc: 0.9734
Epoch 2/5
150/150 [==============================] - 267s 2s/step - loss: 0.0737 - acc: 0.9717 - val_loss: 0.1477 - val_acc: 0.9713
Epoch 3/5
150/150 [==============================] - 266s 2s/step - loss: 0.0604 - acc: 0.9796 - val_loss: 0.2013 - val_acc: 0.9731
Epoch 4/5
150/150 [==============================] - 266s 2s/step - loss: 0.0479 - acc: 0.9810 - val_loss: 0.1519 - val_acc: 0.9713
Epoch 5/5
150/150 [==============================] - 272s 2s/step - loss: 0.0424 - acc: 0.9860 - val_loss: 0.1587 - val_acc: 0.9734


In [32]:
history.history

{'acc': [0.9683181605517442,
  0.9712449199291947,
  0.9797775495683139,
  0.980717181191554,
  0.9858442841855451],
 'loss': [0.08669909199117173,
  0.07407220898237689,
  0.06012557869787128,
  0.04799846998569165,
  0.04265388627777312],
 'val_acc': [0.973360650607797,
  0.9713114688142401,
  0.9731404906954647,
  0.9713114710127722,
  0.973360650607797],
 'val_loss': [0.13408325952535008,
  0.14774908389148042,
  0.20129988386756587,
  0.15186736612366106,
  0.15870190767636216]}

In [33]:
os.chdir("C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/")
model.save('jump_nojump_v3.h5')

In [18]:
os.chdir("C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/")
model = load_model('jump_nojump_v1.h5')

In [34]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(250, 75),
        batch_size=221,
        class_mode='binary',
        shuffle = False)


Found 221 images belonging to 2 classes.


In [35]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

test acc: 0.9502262473106384


In [36]:
preds = model.predict_generator(test_generator, steps = 1)

In [37]:
preds_r = preds.round()
np.unique(preds_r, return_counts=True)

(array([0., 1.], dtype=float32), array([ 17, 204], dtype=int64))

In [38]:
preds_r[1:25]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.]], dtype=float32)

In [39]:
test_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])

In [29]:
file = "data_small/test/jump/direction_up171.png"

In [40]:
start = time.time()
file = "data_small/test/jump/direction_up338.png"
img = image.load_img(file, target_size = (250, 75))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

model.predict(img_tensor)[0,0]
end = time.time()
print(end - start)

0.04586172103881836
